In [ ]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:

!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My\ Drive/QGs/train/

/content/drive/My Drive/QGs/train


In [ ]:
import argparse
import time
import torch_geometric
import numpy as np
import scipy.sparse as sp
import torch
from torch import optim
from torch_geometric.data import Data

from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset, DataLoader
from itertools import cycle, islice
from torch_geometric.utils import dropout_adj


In [ ]:
from optimizer import loss_function
import iterableJet
from iterableJet import IterableMuons
import class_jet
from class_jet import FCMuonsGPU
from model import GraphPooling

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=100, help='Initial learning rate.') #100

parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.4, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [ ]:
samples=torch.load('../datasets/graphs_100k.pt')#np.load('../datasets/QG_jets.npz')

In [ ]:
samples[0]['x']

In [ ]:
for sam in samples:
  sam['x'][:,:3]=1/sam['x'][:,:3]
  sam['x'][:,3]=50/sam['x'][:,3]

In [ ]:
samples[0]['x']

In [ ]:

model = GraphPooling(in_channels=4, out_channels1=64, out_channels2=128, out_channels3=256,out_channels4=128,out_channels5=64,dropout=args.dropout,
                    batch_size=args.batch_size)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.001)
model.train()
model.to(device)

GraphPooling(
  (sage1): SAGEConv(4, 64)
  (sage2): SAGEConv(64, 128)
  (sage3): SAGEConv(128, 256)
  (sage4): SAGEConv(256, 128)
  (tr1): Linear(in_features=128, out_features=64, bias=True)
  (tr2): Linear(in_features=64, out_features=1, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
  (bano1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (edge1): EdgePooling(64)
  (edge2): EdgePooling(128)
  (edge3): EdgePooling(256)
)

In [ ]:
def sizer(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

In [ ]:

sizes=sizer(samples)
fin=[]
countit=0
for m in sizes:
    fin.append(np.repeat(countit,m))
    countit+=1


In [ ]:

from torch.autograd import Variable
import torch_geometric.transforms
from torch_geometric.transforms import RadiusGraph
from torch_geometric.transforms import knn_graph
import torch_geometric.data
from torch_geometric.data import Data
from scipy.spatial import distance_matrix
from torch_geometric.nn import knn_graph

In [ ]:
def getVec(sample):
  fin=[]
  countit=0
  for m in sample:
      fin.append(np.repeat(countit,m))
      countit+=1
  return fin

In [ ]:
from torch_geometric.data import DataLoader
#jet=IterableMuons(samples)
loader = DataLoader(samples, batch_size=args.batch_size,num_workers=0,pin_memory=True)

In [ ]:
for epoch in range(50):
    count=0
    c1,c2=0,args.batch_size
    epLoss=0
    t = time.time()
    for el in islice(loader,0,1000):
        
        gra=el.x
        adj=el.edge_index

        check=torch.LongTensor(np.hstack(np.array(getVec(sizes[c1:c2]))))

        count+=1
        
        gra=gra.to(device)
        adj=adj.to(device)
        lengs=check.to(device)

        optimizer.zero_grad()
       
        r1 = model(gra,adj,lengs)
        loss = loss_function(r1.float(),el.y.float().unsqueeze_(1).cuda())      

        loss.backward()

        optimizer.step()

        cur_loss = loss.item()
        
        epLoss+=float(cur_loss)

        c1+=args.batch_size
        c2+=args.batch_size

        if count%500==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()


    if epoch%2==0:
      torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch':epoch,
            'loss': loss,
            'epLoss':epLoss
            }, './edge_mse_august.pth')

           

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0001 train_loss= 0.67718 time= 114.48943
Epoch: 0001 train_loss= 0.65986 time= 113.50515
Epoch: 0002 train_loss= 0.62434 time= 113.41038
Epoch: 0002 train_loss= 0.62335 time= 110.00434
Epoch: 0003 train_loss= 0.60420 time= 111.11023
Epoch: 0003 train_loss= 0.59683 time= 110.01235
Epoch: 0004 train_loss= 0.59071 time= 108.87147
Epoch: 0004 train_loss= 0.58374 time= 109.93074
Epoch: 0005 train_loss= 0.57645 time= 113.08891
Epoch: 0005 train_loss= 0.57464 time= 111.44581
Epoch: 0006 train_loss= 0.56915 time= 114.24872
Epoch: 0006 train_loss= 0.56567 time= 111.85716
Epoch: 0007 train_loss= 0.57092 time= 107.80062
Epoch: 0007 train_loss= 0.57032 time= 107.35260
Epoch: 0008 train_loss= 0.56714 time= 109.54717
Epoch: 0008 train_loss= 0.56612 time= 108.12490
Epoch: 0009 train_loss= 0.56709 time= 110.89400
Epoch: 0009 train_loss= 0.56412 time= 109.45017
Epoch: 0010 train_loss= 0.56805 time= 106.72144
Epoch: 0010 train_loss= 0.56307 time= 106.66702
Epoch: 0011 train_loss= 0.56581 time= 10

KeyboardInterrupt: ignored